## Importing necessary modules

In [ ]:
import os
import io
import glob
import time
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from itertools import product

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap

from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Random seed

In [ ]:
def same_seeds(seed):
    # Python built-in random module
    # random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    # qiskit
    algorithm_globals.random_seed = seed

RANDOM_SEED = 0
same_seeds(RANDOM_SEED)

## Directory paths

In [ ]:
# Define the directory paths for the results
sub_dirname = '4PCs_QNN_KFold_best_loss'
result_dir = os.path.join("./results", sub_dirname)
results_dir = {
    'figures': os.path.join(result_dir, 'figures'),
    'logs': os.path.join(result_dir, 'logs'),
    'hyperparameters': os.path.join(result_dir, 'hyperparameters'),
    'metrics': os.path.join(result_dir, 'metrics'),
    'checkpoints': os.path.join(result_dir, 'checkpoints'),
}

# Remove the existing directories and create new ones
for key in results_dir.keys():
    if isinstance(results_dir[key], dict):
        for k in results_dir[key].keys():
            if os.path.exists(results_dir[key][k]):
                shutil.rmtree(results_dir[key][k])
            os.makedirs(results_dir[key][k])
    else:
        if os.path.exists(results_dir[key]):
            shutil.rmtree(results_dir[key])
        os.makedirs(results_dir[key])

## Load the 4PCs dataset

In [ ]:
# load the dataset
# feature_1, feature_2, feature_3, feature_4, label
train_data_path = './data/X_train__w_4PCs.xlsx'
test_data_path = './data/X_test_w_4PCs.xlsx'
train_data = pd.read_excel(train_data_path)
test_data = pd.read_excel(test_data_path)

# rename the columns
columns = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'label']
train_data.columns = columns
test_data.columns = columns

# display the dataset information and statistics
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print("-"*50)

print(f"Train data class distribution: \n{train_data['label'].value_counts()}")
print("-"*50)
print(f"Test data class distribution: \n{test_data['label'].value_counts()}")
print("-"*50)

print(f"Train data description: \n{train_data.describe()}")
print(f"Test data description: \n{test_data.describe()}")

In [ ]:
# visualize the train data
print(f'Train data: {train_data.shape}')
sns.pairplot(train_data, hue="label")
plt.show()

In [ ]:
# visualize the test data
print(f'Test data: {test_data.shape}')
sns.pairplot(test_data, hue='label')
plt.show()

## Models

### Example of Parameters

In [ ]:
# Example of Parameters
num_qubits_example = 4
feature_map_reps_example = 2
ansatz_reps_example = 2
loss_fn_example = 'cross_entropy'
optimizer_example = COBYLA(maxiter=100)

### Interpret Method

In [ ]:
# parity maps bitstrings to 0 or 1
def parity(x):
    return "{:b}".format(x).count("1") % 2

### VQC Model

In [ ]:
from qiskit.primitives import Sampler

def create_VQC_model(num_qubits, feature_map_reps, ansatz_reps, loss_fn, optimizer, callback):
    # Create the feature map and ansatz
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=feature_map_reps)
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_reps)
    # Create the sampler
    sampler = Sampler()

    # Create the VQC model
    vqc = VQC(
        sampler=sampler,
        feature_map=feature_map,
        ansatz=ansatz,
        loss=loss_fn,
        optimizer=optimizer,
        callback=callback,
    )

    return vqc

In [ ]:
# Example of the VQC model
circuit = create_VQC_model(
    num_qubits=num_qubits_example,
    feature_map_reps=feature_map_reps_example,
    ansatz_reps=ansatz_reps_example,
    loss_fn=loss_fn_example,
    optimizer=optimizer_example,
    callback=None
).circuit
circuit.decompose().draw(output="mpl", style="clifford", fold=20)

### SamplerQNN Model

In [ ]:
from qiskit.primitives import Sampler

def create_SamplerQNN_model(num_qubits, feature_map_reps, ansatz_reps, loss_fn, optimizer, callback):
    # Create the feature map and ansatz
    feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=feature_map_reps)
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_reps)

    # Create the quantum circuit
    qc = QuantumCircuit(num_qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # Create the sampler
    sampler = Sampler()

    # Define the output shape
    output_shape = 2 # parity maps bitstrings to 0 or 1

    # Create the QNN model
    sampler_qnn = SamplerQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        interpret=parity,
        output_shape=output_shape,
        input_gradients=False
    )

    classifier = NeuralNetworkClassifier(
        neural_network=sampler_qnn,
        optimizer=optimizer,
        callback=callback,
        loss=loss_fn,
        one_hot=True,
    )

    return classifier

In [ ]:
# Example of SampleQNN model
circuit = create_SamplerQNN_model(
    num_qubits=num_qubits_example,
    feature_map_reps=feature_map_reps_example,
    ansatz_reps=ansatz_reps_example,
    loss_fn=loss_fn_example,
    optimizer=optimizer_example,
    callback=None
).neural_network.circuit
circuit.decompose().draw(output="mpl", style="clifford", fold=20)

### SamplerQNN Model with Custom FeatureMap and Ansatz

In [ ]:
from qiskit.primitives import Sampler

def custom_feature_map(num_qubits, reps):
    inputs = ParameterVector("input", num_qubits)
    qc = QuantumCircuit(num_qubits)

    for i in range(reps):
        for j in range(num_qubits):
            qc.h(j)
            qc.ry(inputs[j], j)

    return qc

def custom_ansatz(num_qubits, reps):
    weights = ParameterVector("weight", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)

    for i in range(reps):
        for j in range(num_qubits-1):
            qc.cx(j, j + 1)
        qc.cx(num_qubits-1, 0)
        qc.barrier()
        for j in range(num_qubits):
            qc.ry(weights[i*num_qubits + j], j)

    return qc

def create_custom_SamplerQNN_model(num_qubits, feature_map_reps, ansatz_reps, loss_fn, optimizer, callback):
    # Create the feature map and ansatz
    feature_map = custom_feature_map(num_qubits, feature_map_reps)
    ansatz = custom_ansatz(num_qubits, ansatz_reps)

    # Create the quantum circuit
    qc = QuantumCircuit(num_qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # Create the sampler
    sampler = Sampler()

    # Define the output shape
    output_shape = 2 # parity maps bitstrings to 0 or 1

    # Create the QNN model
    sampler_qnn = SamplerQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        interpret=parity,
        output_shape=output_shape,
        input_gradients=False
    )

    # Create the classifier
    classifier = NeuralNetworkClassifier(
        neural_network=sampler_qnn,
        optimizer=optimizer,
        callback=callback,
        loss=loss_fn,
        one_hot=True,
    )

    return classifier

In [ ]:
# Example of the custom model
circuit = create_custom_SamplerQNN_model(
    num_qubits=num_qubits_example,
    feature_map_reps=feature_map_reps_example,
    ansatz_reps=ansatz_reps_example,
    loss_fn=loss_fn_example,
    optimizer=optimizer_example,
    callback=None
).neural_network.circuit
circuit.decompose().draw(output="mpl", style="clifford", fold=20)

## Analysis of Validation Metrics

In [ ]:
from typing import List

def compute_confusion_matrix_values(confusion):
    """
    Compute the TP, FP, FN, and TN values from the confusion matrix.
    Parameters:
        confusion (np.ndarray): Confusion matrix of shape (n_classes, n_classes).
    Returns:
        tuple: Tuple containing TP, FP, FN, and TN values.
    """
    if confusion.shape == (2, 2):
        """
        [
            [TN, FP]
            [FN, TP]
        ]
        """
        TN, FP, FN, TP = confusion.ravel()
    else:
        # True Positive: The number of samples that the model correctly predicts as class i
        TP = np.diag(confusion)
        # False Positive: The number of samples that the model incorrectly predicts as class i
        FP = confusion.sum(axis=0) - TP
        # False Negative: The number of samples that the model incorrectly predicts as not class i
        FN = confusion.sum(axis=1) - TP
        # True Negative: The number of samples that the model correctly predicts as not class i
        TN = confusion.sum() - (TP + FP + FN)
    return TP, FP, FN, TN

def calculate_metrics(confusion, average=None, weighted=None):
    """
    Calculate various classification metrics based on the provided confusion matrix.
    Parameters:
        confusion (np.ndarray): Confusion matrix of shape (n_classes, n_classes).
        average (str, optional): Type of averaging to be performed on the data.
                                Options are 'micro', 'macro', 'weighted', or None. Default is None.
        weighted (list or np.ndarray, optional): Weights for each class if average is 'weighted'.
                                                Must be provided if average is 'weighted'.
    Returns:
        dict: Dictionary containing the calculated metrics.
    """
    # Check the input arguments
    if isinstance(confusion, List):
        confusion = np.array(confusion) # Convert the list to a numpy array
    if not isinstance(confusion, np.ndarray):
        raise TypeError("The confusion matrix must be a numpy array.")
    if len(confusion.shape) != 2 or confusion.shape[0] != confusion.shape[1]:
        raise ValueError("The confusion matrix must be a square matrix.")
    if average not in [None, 'macro', 'micro', 'weighted']:
        raise ValueError("The average must be one of 'micro', 'macro', 'weighted', or None.")
    if average == 'weighted' and weighted is None:
        raise ValueError("The weights must be provided if average is 'weighted'.")
    if weighted is not None and len(confusion) != len(weighted):
        raise ValueError("The number of classes and the length of the weights must be the same.")

    # Compute the TP, FP, FN, and TN values
    TP, FP, FN, TN = compute_confusion_matrix_values(confusion)

    # If the average is 'micro', sum the values
    if average == 'micro':
        TP, FP, FN, TN = map(np.sum, [TP, FP, FN, TN])

    def safe_divide(numerator, denominator):
        """Safely divide two numbers, returning 0 if the denominator is 0."""
        return np.divide(numerator, denominator, out=np.zeros_like(numerator, dtype=float), where=denominator!=0)

    # Initialize the metrics dictionary
    metrics = {}

    # Accuracy (ACC)
    accuracy = safe_divide(TP + TN, TP + TN + FP + FN)
    # Precision or Positive Predictive Value (PPV)
    precision = safe_divide(TP, TP + FP)
    # Recall or Sensitivity or True Positive Rate (TPR)
    recall = safe_divide(TP, TP + FN)
    # F1 Score
    f1 = safe_divide(2 * (precision * recall), precision + recall)
    # Matthews Correlation Coefficient (MCC)
    mcc = safe_divide((TP * TN - FP * FN), np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)))
    # Negative Predictive Value (NPV)
    npv = safe_divide(TN, TN + FN)
    # True Negative Rate (TNR) or Specificity (SPC)
    tnr = safe_divide(TN, TN + FP)
    # False Positive Rate (FPR)
    fpr = safe_divide(FP, TN + FP)
    # False Discovery Rate (FDR)
    fdr = safe_divide(FP, TP + FP)
    # False Negative Rate (FNR)
    fnr = safe_divide(FN, TP + FN)

    # Create a dictionary of all metrics
    metrics = {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'TPR': recall,
        'SPC': tnr,
        'PPV': precision,
        'F1': f1,
        'ACC': accuracy,
        'NPV': npv,
        'FPR': fpr,
        'FDR': fdr,
        'FNR': fnr,
        'MCC': mcc
    }

    if average == 'macro':
        for metric, values in metrics.items():
            metrics[metric] = np.mean(values)
    elif average == 'weighted':
        for metric, values in metrics.items():
            metrics[metric] = np.average(values, weights=weighted)

    return metrics

## Hyperparameters

In [ ]:
# Define the hyperparameters
target_names = ['Class 0', 'Class 1']
k_folds = 5 # K-fold cross-validation
optimizer_fn = COBYLA
loss_fn = 'cross_entropy'
average = 'macro'

In [ ]:
# Define the search space for the hyperparameters
models = ['SamplerQNN']
epochs = [50, 100, 200]
feature_map_reps_list = [2, 3]
ansatz_reps_list = [2, 3]

In [ ]:
# Function to create the model based on the parameters
def get_create_model_function(model_name):
    if model_name == 'VQC':
        return create_VQC_model
    elif model_name == 'SamplerQNN':
        return create_SamplerQNN_model
    elif model_name == 'CustomSamplerQNN':
        return create_custom_SamplerQNN_model
    else:
        raise ValueError(f"Invalid model name: {model_name}")

In [ ]:
# Create the parameter combinations
param_combinations = list(product(models, epochs, feature_map_reps_list, ansatz_reps_list))

print(f"Number of parameter combinations: {len(param_combinations)}")
for i, params in enumerate(param_combinations):
    model_name, epochs, feature_map_reps, ansatz_reps = params
    print(f"Parameter combination {i+1}:")
    print(f"\tModel: {model_name}")
    print(f"\tEpochs: {epochs}")
    print(f"\tFeature map reps: {feature_map_reps}")
    print(f"\tAnsatz reps: {ansatz_reps}")

## Stratified K-Fold Cross Validation
Find the best parameter combinations

In [ ]:
# Split the data into features and labels
x_train = train_data.iloc[:, :-1].values
y_train = train_data['label'].values
x_test = test_data.iloc[:, :-1].values
y_test = test_data['label'].values

In [ ]:
objective_func_vals = [] # To store the objective function values
def callback_recorder(weights, obj_func_eval):
        objective_func_vals.append(obj_func_eval)

In [ ]:
# Initialize the StratifiedKFold
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=RANDOM_SEED)

# Iterate over the parameter combinations
total_start = time.time() # To store the total time taken for the entire training
mean_losses = [] # To store the mean losses for each parameter combination
metrics_columns = [] # To store the columns for the metrics
metrics_record = [] # To store the metrics for each parameter combination
for param_index, param_comb in enumerate(param_combinations):
    # Get the parameter combination
    model_name, epochs, feature_map_reps, ansatz_reps = param_comb
    create_model_fn = get_create_model_function(model_name)
    optimizer = optimizer_fn(maxiter=epochs)

    print("="*100)
    print(f"Parameter combination {param_index+1}/{len(param_combinations)}")
    print(f"Model: {model_name}, Epochs: {epochs}, Feature map reps: {feature_map_reps}, Ansatz reps: {ansatz_reps}")
    print("="*100)

    # Save the hyperparameters configuration to a file
    params_names = f'{model_name}_fmr{feature_map_reps}_ar{ansatz_reps}'
    with open(os.path.join(results_dir['hyperparameters'], f'hyperparameters_{params_names}.txt'), 'w') as f:
        f.write(f"Optimizer: {optimizer.__class__.__name__}\n")
        f.write(f"Loss function: {loss_fn}\n")
        f.write(f"Model: {model_name}\n")
        f.write(f"Epochs: {epochs}\n")
        f.write(f"Feature map reps: {feature_map_reps}\n")
        f.write(f"Ansatz reps: {ansatz_reps}\n")

    # Create the classifier model
    num_qubits = x_train.shape[1]
    classifier = create_model_fn(num_qubits, feature_map_reps, ansatz_reps, loss_fn, optimizer, callback_recorder)

    # Save the circuit diagram
    classifier_circuit = None
    if hasattr(classifier, 'circuit'):
        classifier_circuit = classifier.circuit
    else:
        classifier_circuit = classifier.neural_network.circuit
    classifier_circuit.decompose().draw(output="mpl", style="clifford", fold=20) \
        .savefig(os.path.join(results_dir['figures'], f'circuit_{params_names}.png'))

    # Iterate over the folds
    fold_losses = [] # To store the losses for each fold
    for fold_index, (train_index, val_index) in enumerate(skf.split(x_train, y_train)):
        fold_time_start = time.time() # To store the time taken for the current parameter combination

        # Display the fold number
        print(f"Fold {fold_index+1}:")

        # Define the model name
        fold_name = f"{params_names}_fold{fold_index+1}"

        # ================================================================================
        # Data Preprocessing
        # ================================================================================
        # Split the data into training and validation sets
        x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Normalize the data
        scaler = MinMaxScaler()
        x_train_fold = scaler.fit_transform(x_train_fold)
        x_val_fold = scaler.transform(x_val_fold)

        # ================================================================================
        # Model Training
        # ================================================================================
        # Train the model
        objective_func_vals = [] # Reset the objective function values
        classifier.fit(x_train_fold, y_train_fold)

        # Save the model
        classifier.save(os.path.join(results_dir['checkpoints'], f'{fold_name}.model'))

        # Plot the training loss curve, and save the figure
        plt.figure(figsize=(6, 4))
        plt.title(f"Objective function value against iteration - {fold_name}")
        plt.xlabel("Iteration")
        plt.ylabel("Objective function value")
        plt.plot(range(len(objective_func_vals)), objective_func_vals)
        plt.savefig(os.path.join(results_dir['figures'], f'loss_curve_{fold_name}.png'))
        plt.show()
        plt.close()

        # Save the loss values to a file
        with open(os.path.join(results_dir['logs'], f'loss_{fold_name}.txt'), 'w') as f:
            f.write(",".join(map(str, objective_func_vals)))

        # Record the final loss value for the fold
        fold_losses.append(objective_func_vals[-1])
        print(f"Final loss: {objective_func_vals[-1]}")

        # ================================================================================
        # Model Evaluation
        # ================================================================================
        # Evaluate score on the validation data
        val_score = classifier.score(x_val_fold, y_val_fold)
        print(f"Validation score: {val_score}")

        # Calculate and plot the confusion matrix for the validation data
        y_pred_val = classifier.predict(x_val_fold)
        confusion = confusion_matrix(y_val_fold, y_pred_val)
        df_cm = pd.DataFrame(confusion, index=target_names, columns=target_names)
        plt.figure(figsize=(6, 4))
        sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Prediction")
        plt.ylabel("Ground Truth")
        plt.title(f"Confusion Matrix - {fold_name}")
        plt.savefig(os.path.join(results_dir['figures'], f'confusion_matrix_{fold_name}.png'))
        plt.show()

        # Calculate the classification report
        print("Classification report:")
        print(classification_report(y_val_fold, y_pred_val, target_names=target_names, zero_division=0))

        # Calculate the metrics
        metrics = calculate_metrics(confusion, average=average)
        metrics = {k: np.round(v, 4) for k, v in metrics.items()} # Round the values to 4 decimal places

        # Display the metrics
        print(f"Averaging method: {average}")
        print("Metrics:")
        for metric, value in metrics.items():
            print(f"\t{metric}: {value}")

        # Save the parameters and metrics to a file
        metrics_columns = ['Model Name', 'Epochs', 'Feature Map Reps', 'Ansatz Reps', 'Fold', 'Loss'] + list(metrics.keys())
        metrics_values = [model_name, epochs, feature_map_reps, ansatz_reps, fold_index+1, objective_func_vals[-1]] + list(metrics.values())
        metrics_record.append(metrics_values)

        # Calculate the time taken for the current fold
        fold_time_elapsed = time.time() - fold_time_start
        print(f"Fold time: {round(fold_time_elapsed)} seconds")

    # ================================================================================
    # Current parameter combination training is done
    # Record the mean validation score for the parameter combination
    # ================================================================================

    # Calculate the mean validation score for the parameter combination
    mean_loss = np.array(fold_losses).mean()
    mean_losses.append(mean_loss)
    print(f"Mean validation loss: {mean_loss}")

# ================================================================================
# All parameter combinations training is done
# ================================================================================
total_elapsed = time.time() - total_start
print(f"Total training time: {round(total_elapsed)} seconds")

# Save the metrics to a file
metrics_df = pd.DataFrame(metrics_record, columns=metrics_columns)
metrics_df.to_csv(os.path.join(results_dir['metrics'], 'all_metrics.csv'), index=False)

In [ ]:
# Select the best parameter combination
best_param_index = np.argmin(mean_losses)
best_param_comb = param_combinations[best_param_index]
best_model_name, best_epochs, best_feature_map_reps, best_ansatz_reps = best_param_comb
best_create_model_fn = get_create_model_function(best_model_name)

print(f"Best model: {best_model_name}")
print(f"Best epochs: {best_epochs}")
print(f"Best feature map reps: {best_feature_map_reps}")
print(f"Best ansatz reps: {best_ansatz_reps}")
print(f"Best mean loss: {mean_losses[best_param_index]}")

In [ ]:
# Save the besthyperparameters configuration to a file
best_model_name = f'best_{best_model_name}_fmr{best_feature_map_reps}_ar{best_ansatz_reps}'
with open(os.path.join(results_dir['hyperparameters'], f'hyperparameters_{best_model_name}.txt'), 'w') as f:
    f.write(f"Optimizer: {optimizer.__class__.__name__}\n")
    f.write(f"Loss function: {loss_fn}\n")
    f.write(f"Model: {model_name}\n")
    f.write(f"Epochs: {epochs}\n")
    f.write(f"Feature map reps: {feature_map_reps}\n")
    f.write(f"Ansatz reps: {ansatz_reps}\n")

## Train the Model with Best Parameter Combinations

### Data Preprocessing

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

### Model Training

In [ ]:
from IPython.display import clear_output

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

# callback function that draws a live plot when the .fit() method is called
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
# Create the classifier model
num_qubits = x_train.shape[1]
objective_func_vals = [] # Reset the objective function values
classifier = best_create_model_fn(num_qubits, best_feature_map_reps, best_ansatz_reps, loss_fn, optimizer, callback_graph)

# Display the circuit
classifier_circuit = None
if hasattr(classifier, 'circuit'):
    classifier_circuit = classifier.circuit
else:
    classifier_circuit = classifier.neural_network.circuit

classifier_circuit.decompose().draw(output="mpl", style="clifford", fold=20) \
    .savefig(os.path.join(results_dir['figures'], f'circuit_{best_model_name}.png'))

In [ ]:
# fit the model to the training data
start = time.time()
classifier.fit(x_train, y_train)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

In [ ]:
# Save the model
classifier.save(os.path.join(results_dir['checkpoints'], f'{best_model_name}.model'))

## Performance

In [ ]:
# Load the best model
best_model_path = os.path.join(results_dir['checkpoints'], f'{best_model_name}.model')
classifier = NeuralNetworkClassifier.load(best_model_path)

In [ ]:
# evaluate the model on the training and testing data
train_accuracy = classifier.score(x_train, y_train)
test_accuracy = classifier.score(x_test, y_test)
print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

In [ ]:
# predict labels
y_predict = classifier.predict(x_test)

# visualize results
valid_df = pd.DataFrame(x_test, columns=columns[:-1])
valid_df["is_correct"] = y_test == y_predict
sns.pairplot(valid_df, hue="is_correct", vars=valid_df.columns[:-1])
plt.savefig(os.path.join(results_dir['figures'], f'pairplot_{best_model_name}.png'))
plt.show()

In [ ]:
# compute confusion matrix
target_names = ["Class 0", "Class 1"]
confusion = confusion_matrix(y_test, y_predict)
df_cm = pd.DataFrame(confusion, index=target_names, columns=target_names)

# plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Prediction")
plt.ylabel("Ground Truth")
plt.title("Confusion Matrix")
plt.savefig(os.path.join(results_dir['figures'], f'confusion_matrix_{best_model_name}.png'))
plt.show()

In [ ]:
# Performance metrics
print("Classification report:")
print(classification_report(y_test, y_predict, target_names=target_names, zero_division=0))

In [ ]:
average = None
metrics = calculate_metrics(confusion, average=average)
metrics = {k: np.round(v, 4) for k, v in metrics.items()} # Round the values to 4 decimal places

print(f"Averaging method: {average}")
metrics_df = pd.DataFrame(metrics, index=target_names)
print(metrics_df.T) # transpose

In [ ]:
average = 'macro'
metrics = calculate_metrics(confusion, average=average)
metrics = {k: np.round(v, 4) for k, v in metrics.items()} # Round the values to 4 decimal places

# Display the performance metrics
print(f"Averaging method: {average}")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

## Save Metrics to CSV

In [ ]:
# Save the parameters and metrics to a CSV file
columns = ['Model Name', 'Epochs', 'Feature Map Reps', 'Ansatz Reps'] + list(metrics.keys())
data = [model_name, epochs, best_feature_map_reps, best_ansatz_reps] + list(metrics.values())
metrics_df = pd.DataFrame([data], columns=columns)
metrics_df.to_csv(os.path.join(results_dir['metrics'], f'metrics_{model_name}.csv'), index=False)
print(metrics_df)